## Fuel Uncertainity Risk Management
The purpose of this note book is to illustrate how to define fuel price risk as a contraint in an optimization problem.

In [1]:
import pandas as pd
import numpy as np
from pulp import *
from scipy.stats import multivariate_normal as mn
from scipy.stats import norm
from scipy import optimize
from random import random
import time

In [2]:
path = 'c:\\users\\mark113\\desktop\\jupyter\\shell\\'
demand = pd.read_csv(path + 'dataset\\demand.csv')
fuels = pd.read_csv(path + 'dataset\\fuels.csv')
cost_profiles = pd.read_csv(path + 'dataset\\cost_profiles.csv')
vehicles = pd.read_csv(path + 'dataset\\vehicles.csv')
vehicles_fuels = pd.read_csv(path + 'dataset\\vehicles_fuels.csv')
carbon_df = pd.read_csv(path + 'dataset\\carbon_emissions.csv')

### Convert demand and range to dictionaries
In order to be fast, Demand and Range has to be expressed as a vector

In [3]:
available_years = []
size_map = []
distance_map = []
fuel_map = []
demand_vector = []
range_vector = []
range_consumption_vector= []
fuel_price_vector = []
fuel_price_consumption_vector = []
vf = vehicles.merge(vehicles_fuels, on='ID')
vf = vf.merge(fuels, on=['Fuel', 'Year'])
variable_ID = []
vehicle_map = []
for size in ['S1', 'S2', 'S3', 'S4']:    
    for distance in ['D1', 'D2', 'D3', 'D4']:
        demand_vector = demand_vector + [int(m) for m in demand[(demand['Distance'] == distance) & (demand['Size']==size)][['Year','Demand (km)']].groupby('Year').sum()['Demand (km)']]
        subgroup = vf[(vf['Distance'] == distance) & (vf['Size']==size)]
        
        available_years = available_years + subgroup['Year'].tolist()
        size_map = size_map + [size] * subgroup.shape[0]
        distance_map = distance_map + [distance] * subgroup.shape[0]
        fuel_map = fuel_map + subgroup['Fuel'].tolist()
        vehicle_map = vehicle_map + subgroup['Vehicle'].tolist()

        range_vector = range_vector + [int(m) for m in subgroup['Yearly range (km)']]
        fuel_price_vector = fuel_price_vector + [cost for cost in subgroup['Cost ($/unit_fuel)']]
        fuel_price_consumption_vector = fuel_price_consumption_vector +\
        [cost*consumpt*r for cost, consumpt, r in zip(subgroup['Cost ($/unit_fuel)'], 
                                                      subgroup['Consumption (unit_fuel/km)'],
                                                      subgroup['Yearly range (km)'])]
        range_consumption_vector = range_consumption_vector +\
        [consumpt*r for consumpt, r in zip(subgroup['Cost ($/unit_fuel)'], 
                                                 subgroup['Consumption (unit_fuel/km)'])]

In [4]:
class check_demand:
    def __init__(self, d, r, yr_ndx, size_map, distance_map, fuel_map, vehicle_map):
        self.demand_by_size_distance_year = d
        self.range_by_size_distance_year_veh = r
        self.year_index = yr_ndx
        self.size_map = size_map
        self.fuel_map = fuel_map
        self.distance_map = distance_map
        self.vehicle_map = vehicle_map
        self.sizes = ['S1', 'S2', 'S3', 'S4']
        self.distances = ['D1','D2', 'D3','D4']
        
    def group_by_veh_fuel(self, plan):

        # aggregate by vehicle
        supply = {}
        for y in years:
            supply[y] = {}
            for s in self.sizes:
                supply[y][s] = {}
                for d in self.distances:
                    supply[y][s][d] = 0

        for i in range(len(plan)):
            supply[self.year_index[i]][self.size_map[i]][self.distance_map[i]] += plan[i]*self.range_by_size_distance_year_veh[i]

        return [supply[y][s][d] for s in self.sizes for d in self.distances for y in years]
    
    def verify(self, x):
        return -sum([d-max(m, 0) for d, m in zip(self.demand_by_size_distance_year, self.group_by_veh_fuel(x))])

class fuel_risk:
    
    def __init__(self, r, s, pct, range_map, risk_budget, year_map, fuel_map):
        self.risk_map = [0]*len(fuel_map)
        self.range_map = range_map
        BS = {}
        for y in years:
            BS[y] = {}

        for y, f, c in zip(fuels['Year'], fuels['Fuel'], fuels['Cost ($/unit_fuel)']):
            t = y - 2023
            BS[y][f] = pct*c*mn.pdf(np.log(c)*(1-pct) + (r+s**2/2)*t, 0,1) - c*mn.pdf(np.log(c)*(1-pct) - (r+s**2/2)*t, 0,1) 
        ct = 0
        for y, f in zip(year_map, fuel_map):
            self.risk_map[ct] = BS[y][f]
            ct += 1
        self.risk_budget = risk_budget
        
    def risk(self, x):
        miles = [n*r for n, r in zip(x, self.range_map)]
        return -sum([n*b for n, b in zip(miles, self.risk_map)]) + self.risk_budget

    
class objective_function:
    def __init__(self, fuel_prices):
        
        self.fuel_prices=fuel_prices
        
    def fuel_cost(self, x):
        return sum([c*a for c, a in zip(self.fuel_prices, x)])
    

In [5]:
max_year = 2038
years = [y for y in range(2023, max_year+1)]

### For this demo, for simplicity, we just focus on fuel cost uncertainity
The risk budget is expressed as a cost $3,500

In [6]:
risk_free_short_rate = .05
risk_budget = 1000
annual_threshold_factor = 2
implied_volatility = .05

In [7]:
n = len(range_vector)
x0 = [19]*n
demand_constraint = check_demand(demand_vector, range_vector, available_years,
                                size_map, distance_map, fuel_map, vehicle_map)
of = objective_function(fuel_price_consumption_vector)
print("Initial Demand {}".format(demand_constraint.verify(x0)))
rbdo_constraint = fuel_risk(risk_free_short_rate, implied_volatility, annual_threshold_factor, range_consumption_vector, 
                            risk_budget, available_years, fuel_map)
print("Initial Risk {}".format(rbdo_constraint.risk(x0)))

Initial Demand 270938722
Initial Risk 202.34618985857162


In [10]:
solution = optimize.minimize(fun=of.fuel_cost, x0=tuple(x0), tol=.1, method='COBYLA', options={'maxiter':1000},
                             bounds=tuple([(0,None)]*len(x0)),
                             constraints=({'type': 'ineq', 'fun': rbdo_constraint.risk},
                                          {'type': 'ineq', 'fun': demand_constraint.verify}) )

In [11]:
print(solution)
print(" cost {}".format(of.fuel_cost(solution.x)))
print(" risk {}".format(rbdo_constraint.risk(solution.x)))
print(" demand {}".format(demand_constraint.verify(solution.x)))

 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: 36994232.56664027
       x: [ 2.193e-13  1.678e-13 ...  1.409e+01  3.855e-13]
    nfev: 1000
   maxcv: 1.3537704944610596e-05
 cost 36994232.56664027
 risk 989.7118903248237
 demand -1.3537704944610596e-05
